<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Retrieve-Movie-Frames" data-toc-modified-id="Retrieve-Movie-Frames-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Retrieve Movie Frames</a></span></li><li><span><a href="#Extracting-Faces" data-toc-modified-id="Extracting-Faces-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Extracting Faces</a></span></li><li><span><a href="#Classify-Each-face" data-toc-modified-id="Classify-Each-face-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Classify Each face</a></span></li><li><span><a href="#Generate-Overview-for-one-Movie" data-toc-modified-id="Generate-Overview-for-one-Movie-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Generate Overview for one Movie</a></span></li></ul></div>

In [1]:
import sys
sys.path.insert(0,'..')

In [97]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import mtcnn

from cv2 import imread
from cv2 import CascadeClassifier

In [2]:
from diversity_in_cinema.cnn_model import predict_face
from diversity_in_cinema.data import get_images
from diversity_in_cinema.scraper import get_movies

# Retrieve Movie Frames

In [107]:
# choose movie name
movie_list = list(get_movies().keys())
movie_list[420]

'Man of Steel (2013)'

In [108]:
df_movie = get_images(movie_list[420], frame_interval=4)

In [110]:
df_movie.head()

,title,frame_no,img_array
0,Man of Steel (2013),1,"[[[98, 91, 72], [96, 90, 68], [94, 88, 66], [9..."
1,Man of Steel (2013),5,"[[[80, 83, 66], [82, 85, 68], [84, 87, 68], [8..."
2,Man of Steel (2013),9,"[[[67, 71, 57], [66, 70, 56], [69, 73, 59], [6..."
3,Man of Steel (2013),13,"[[[53, 57, 43], [54, 58, 44], [47, 51, 37], [4..."
4,Man of Steel (2013),17,"[[[2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], ..."


In [115]:
df_movie.to_csv("man_of_steel_frames.csv")

# Extracting Faces

In [145]:
def extract_face_mtcnn(dataframe):
    
    """
    Function which given a dataset of frame numbers and frame arrays
    returns a dataframe of faces detected in each frame
    """

    detector = mtcnn.MTCNN()   
    
    face_dict = {}
    
    print("Extracting faces...")
    
    for frames, frame_id in zip(dataframe["img_array"], tqdm(dataframe["frame_no"])):
        # print bounding box for each detected face
        bboxes = detector.detect_faces(frames)
        
        faces_list = []
        
        if len(bboxes) > 0:
            
            # extract all faces in image
            for box in bboxes:
                
                x1, y1, width, height = box["box"]
        
                if (width * height) >= 10_000:
                    x2, y2 = x1 + width, y1 + height
                    face = frames[y1:y2, x1:x2]

                    # resize
                    faces_list.append(face)

        face_dict[frame_id] = faces_list
        
        faces_df = pd.DataFrame(data={"frame":list(face_dict.keys()),
                                      "faces": list(face_dict.values())})
        
    return faces_df

In [146]:
def extract_face_opencv(dataframe):
    
    """
    Function which given a dataset of frame numbers and frame arrays
    returns a dataframe of faces detected in each frame
    """
    

    classifier = CascadeClassifier('haarcascade_frontalface_default.xml')
        
    face_dict = {}
    
    print("Extracting faces...")
    
    for frames, frame_id in zip(dataframe["img_array"], tqdm(dataframe["frame_no"])):
        
        # perform face detection
        bboxes = classifier.detectMultiScale(np.array(frames, dtype='uint8'))
        
        # print bounding box for each detected face

        faces_list = []
        if len(bboxes) > 0:
            # extract all faces in image
            for box in bboxes:

                x1, y1, width, height = box

                if (width * height) >= 10_000:
                    x2, y2 = x1 + width, y1 + height
                    face = frames[y1:y2, x1:x2]

                    # resize
                    faces_list.append(face)

        face_dict[frame_id] = faces_list
        
        faces_df = pd.DataFrame(data={"frame":list(face_dict.keys()),
                                      "faces": list(face_dict.values())})
        
        return faces_df

# Classify Each face 

In [151]:
# every rwo is a frame and every frame has n faces

def classify_faces(dataframe):
    
    """
    A function which takes in a dataframe of frame number and the
    extracted face images in each frame as an array
    
    """
    
    df_list = []
    i = 0
    for frame, faces in zip(dataframe["frame"], tqdm(dataframe["faces"])):

        frame_list = []
        gender_list = []
        race_list = []
        face_id_list = []

        results = predict_face(faces)

        for faces in results.values():
            gender = faces["gender"]
            race = faces["dominant_race"]

            frame_list.append(frame)
            gender_list.append(gender)
            race_list.append(race)

            face_id_list.append(i)

            i += 1

        df = pd.DataFrame(data={"frame_number":frame_list,
                                "face_id":face_id_list,
                                "gender":gender_list,
                                "race":race_list})
        df_list.append(df) 
        
    return pd.concat(df_list)

# Generate Overview for one Movie

In [148]:
faces_df = extract_face_mtcnn(df_movie)
faces_df

Extracting faces...


100%|████████████████████████████████████▉| 3950/3951 [4:49:02<00:04,  4.39s/it]


,frame,faces
0,1,[]
1,5,[]
2,9,"[[[[24 14 5], [24 14 5], [24 14 5], [23 13 ..."
3,13,[]
4,17,[]
...,...,...
3946,15785,"[[[[6 8 7], [5 7 6], [5 7 6], [4 6 5], [4 6 5]..."
3947,15789,"[[[[2 8 8], [2 8 8], [2 8 8], [2 8 8], [2 8 8]..."
3948,15793,"[[[[6 7 9], [6 7 9], [6 7 9], [6 7 9], [6 7 9]..."
3949,15797,"[[[[107 94 78], [112 99 83], [111 95 80]..."


In [152]:
df_classified = classify_faces(faces_df)

Analyzing:   0%|                                          | 0/2 [00:00<?, ?it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  4.35it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  4.35it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  4.53it/s]

Analyzing:  50%|█████████████████                 | 1/2 [00:00<00:00,  2.25it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  4.22it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  4.22it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  4.49it/s]

Action: race:  5

Analyzing:   0%|                                          | 0/3 [00:00<?, ?it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  3.64it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  3.64it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  3.88it/s]

Analyzing:  33%|███████████▎                      | 1/3 [00:00<00:01,  1.93it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  4.08it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  4.08it/s]



Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  4.28it/s]

Analyzing:  67%|██████████████████████▋           | 2/3 [00:00<00:00,  2.03it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  4.44it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  4.44it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  4.25it/s]

Analyzing:   0%|                                          | 0/3 [00:00<?, ?it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  3.97it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  3.97it/s]

Action: race: 10

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]


Analyzing:   0%|                                          | 0/2 [00:00<?, ?it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  2.71it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  2.71it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  3.28it/s]

Analyzing:  50%|█████████████████                 | 1/2 [00:00<00:00,  1.63it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  3.04it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  3.04it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  3.40it/s]

Analyzing:   0%|

...

 98%|██████████████████████████████████████▏| 3871/3951 [20:26<00:34,  2.30it/s]


Analyzing:   0%|                                          | 0/2 [00:00<?, ?it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  4.14it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  4.14it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  4.27it/s]

Analyzing:  50%|█████████████████                 | 1/2 [00:00<00:00,  2.12it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  3.87it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  3.87it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  4.06it/s]

Analyzing:   0%|

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  4.34it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  4.34it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  4.29it/s]

Analyzing:  50%|█████████████████                 | 1/2 [00:00<00:00,  2.13it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  4.34it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  4.34it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  4.21it/s]

Analyzing:   0%|                                          | 0/2 [00:00<?, ?it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender: 

Analyzing:   0%|                                          | 0/2 [00:00<?, ?it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  4.01it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  4.01it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  4.19it/s]

Analyzing:  50%|█████████████████                 | 1/2 [00:00<00:00,  2.08it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  4.55it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  4.55it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  4.43it/s]

Analyzing:   0%|

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  4.35it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  4.35it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  4.43it/s]

Analyzing:   0%|                                          | 0/2 [00:00<?, ?it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender:   0%|                                     | 0/2 [00:00<?, ?it/s]

Action: gender:  50%|██████████████▌              | 1/2 [00:00<00:00,  3.04it/s]

Action: race:  50%|███████████████▌               | 1/2 [00:00<00:00,  3.04it/s]

Action: race: 100%|███████████████████████████████| 2/2 [00:00<00:00,  3.61it/s]

Analyzing:  50%|█████████████████                 | 1/2 [00:00<00:00,  1.79it/s]

Finding actions:   0%|                                    | 0/2 [00:00<?, ?it/s]

Action: gender: 

In [156]:
df_classified.to_csv("man_of_steel_results.csv")